In [1]:
import py_vncorenlp
import pandas as pd
from transformers import AutoTokenizer
import numpy as np
# import numpy as np
import re

c:\Users\ADMIN\anaconda3\envs\QN\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def convert_to_feature(series, tokenizer, max_sequence_length):
    outputs = []
    outputs = np.zeros((len(series), max_sequence_length))
    cls_id = 0
    eos_id = 2
    pad_id = 1
    for idx, row in enumerate(series): 
        input_ids = tokenizer.encode(row)
        if len(input_ids) > max_sequence_length: 
            input_ids = input_ids[:max_sequence_length] 
            input_ids[-1] = eos_id
        else:
            input_ids = input_ids + [pad_id, ]*(max_sequence_length - len(input_ids))
        outputs[idx,:] = np.array(input_ids)
    return outputs
    
def text_cleaner(review):
    review = review.replace('\n', ' ')
    review = review.replace('-', ' ')
    # review = review.replace('.', '')
    review = re.sub("\s\s+" , " ", review)
    # review = re.sub(r'[^\w\s]', '', review)
    review = review.strip()
    # review = review.lower()
    return review

In [2]:
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir="E:\\OneDrive\\Hackathon\\Problem2\\vncorenlp")
tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
a = "một con vịt."
tokenizer.encode(a)

[0, 16, 73, 25758, 13208, 2]

In [4]:
train_df = pd.read_csv('E:\\OneDrive\\Hackathon\\Problem2\\data\\train-problem2.csv')
test_df = pd.read_csv('E:\\OneDrive\\Hackathon\\Problem2\\data\\public-problem2.csv')

In [5]:
label = ['Dịch vụ vui chơi giải trí.', 'Dịch vụ lưu trú.', 'Hệ thống nhà hàng.', 'Dịch vụ ăn uống.', 'Dịch vụ di chuyển.', 'Dịch vụ mua sắm.']

In [6]:
label

['Dịch vụ vui chơi giải trí.',
 'Dịch vụ lưu trú.',
 'Hệ thống nhà hàng.',
 'Dịch vụ ăn uống.',
 'Dịch vụ di chuyển.',
 'Dịch vụ mua sắm.']

In [7]:
train_df.head()

,Review,a0,a1,a2,a3,a4,a5
0,"Bánh rất nhiều tôm to, tôm giòn nằm chễm chệ t...",0,0,0,5,0,0
1,Đi Quy Nhơn cứ lo không có pizza ăn nhưng nhờ ...,0,0,0,5,0,0
2,"các món ăn ở đây không có gì đặc sắc, có lẽ nấ...",0,0,1,1,0,0
3,"Lần thứ 2 quay lại đây, vì giới thiệu quán cà ...",0,0,0,2,0,0
4,Rất khuyên bạn nên ghé thăm quán bar thể thao này,3,0,0,0,0,0


In [8]:
a_col = train_df.select_dtypes('int64').columns.to_list()

In [9]:
train_df[label] = train_df[a_col]

In [25]:
test_df[label] = test_df[a_col]

In [12]:
train_df.head()

,Review,a0,a1,a2,a3,a4,a5,Dịch vụ vui chơi giải trí.,Dịch vụ lưu trú.,Hệ thống nhà hàng.,Dịch vụ ăn uống.,Dịch vụ di chuyển.,Dịch vụ mua sắm.
0,"Bánh rất nhiều tôm to, tôm giòn nằm chễm chệ t...",0,0,0,5,0,0,0,0,0,5,0,0
1,Đi Quy Nhơn cứ lo không có pizza ăn nhưng nhờ ...,0,0,0,5,0,0,0,0,0,5,0,0
2,"các món ăn ở đây không có gì đặc sắc, có lẽ nấ...",0,0,1,1,0,0,0,0,1,1,0,0
3,"Lần thứ 2 quay lại đây, vì giới thiệu quán cà ...",0,0,0,2,0,0,0,0,0,2,0,0
4,Rất khuyên bạn nên ghé thăm quán bar thể thao này,3,0,0,0,0,0,3,0,0,0,0,0


In [13]:
a_col = train_df.select_dtypes('int64').columns.to_list()
x_train = train_df['Review']
y_train = train_df[label].clip(upper=1)

x_test = test_df['Review']
y_test = test_df[label].clip(upper=1)

In [24]:
data = []
labels = []
count = 0
for index, row in train_df.iterrows():
    for i in label:
        text = row['Review'] + ' ' + i
        data.append(text)
        labels.append(int(row[i] > 0))
    if count == 1:
        break
    # count += 1 
print(len(data))
print(len(labels))

17790
17790


In [30]:
data = []
labels = []
count = 0
for index, row in test_df.iterrows():
    for i in label:
        text = row['Review'] + ' ' + i
        data.append(text)
        labels.append(int(row[i] > 0))
    if count == 1:
        break
    # count += 1 
print(len(data))
print(len(labels))

5934
5934


In [31]:
df = pd.DataFrame(list(zip(data, labels)),
               columns =['Review', 'Label'])

In [32]:
df.head()

,Review,Label
0,"Nơi đó thật kinh khủng, chúng tôi nhìn thấy nh...",0
1,"Nơi đó thật kinh khủng, chúng tôi nhìn thấy nh...",0
2,"Nơi đó thật kinh khủng, chúng tôi nhìn thấy nh...",1
3,"Nơi đó thật kinh khủng, chúng tôi nhìn thấy nh...",1
4,"Nơi đó thật kinh khủng, chúng tôi nhìn thấy nh...",0


In [33]:
df.to_csv('E:\\Git\\Problem2\\data\\test_preprocess.csv')

In [14]:
a = [12,1,2,3,4,5,6,7]
b = a[len(a)-2:]

In [16]:
b.append(0)

In [23]:
import numpy as np
a = np.load('data/target_process.npy')

In [24]:
a.shape

(23724, 1)

In [12]:
import pandas as pd
train_df = pd.read_csv("E:/Git/Problem2/data/train_preprocess.csv")
test_df = pd.read_csv("E:/Git/Problem2/data/test_preprocess.csv")

In [13]:
train_df = train_df.drop(columns='Index')
test_df = test_df.drop(columns='Index')

In [16]:
train_df.to_csv('data/train.csv', index = False)
test_df.to_csv('data/test.csv', index = False)

In [1]:
import numpy as np
data_npy = np.load('data/data.npy')

In [4]:
data_npy.astype(np.int64).shape

(3954, 256)